In [1]:
import os, sys

# Force notebook to run as if cwd = project root
PROJECT_ROOT = "/Users/jayklarin/__DI/Repositories/FaceStats"
os.chdir(PROJECT_ROOT)
print("cwd:", os.getcwd())

# Add src/ to Python import path
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
sys.path.insert(0, SRC_PATH)
print("src path added:", SRC_PATH)


cwd: /Users/jayklarin/__DI/Repositories/FaceStats
src path added: /Users/jayklarin/__DI/Repositories/FaceStats/src


In [2]:
from embeddings.embed_clip import extract_clip_embeddings

INPUT_DIR = "data/processed/preproc"
OUTPUT_FILE = "data/processed/embeddings_clip.parquet"

emb_df = extract_clip_embeddings(
    input_dir=INPUT_DIR,
    output_path=OUTPUT_FILE,
    model_name="openai/clip-vit-base-patch32"
)

emb_df.head()

/opt/anaconda3/envs/facestats/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Saved 80 embeddings → data/processed/embeddings_clip.parquet


filename,embedding
str,list[f64]
"""SFHQ_pt4_00000208.jpg""","[0.050459, -0.042498, … -0.011308]"
"""SFHQ_pt4_00002966.jpg""","[-0.00345, -0.025553, … -0.002623]"
"""SFHQ_pt4_00003463.jpg""","[0.049876, -0.015998, … 0.02997]"
"""SFHQ_pt4_00002219.jpg""","[0.050927, -0.002582, … 0.014047]"
"""SFHQ_pt4_00003112.jpg""","[0.021666, -0.005994, … -0.009105]"
